In [ ]:
import numpy as np
import hls4ml
import tensorflow as tf
import keras
import qkeras


In [ ]:
X_train_val = np.load("../dataset/x_train_set_4x.npy")
X_test = np.ascontiguousarray(np.load("../dataset/x_test_set_4x.npy"))
y_train_val = np.load("../dataset/y_train_set_4x.npy")
y_test = np.load("../dataset/y_test_set_4x.npy")
classes = ["acf_alv", "acf_chap", "reboco", "tijolo_alv", "tijolo_chap"]

# load model

In [ ]:
from qkeras.utils import _add_supported_quantized_objects, load_qmodel


model_path = "/home/webphy/Desktop/dnn_processor/hls4ml/saved_qmodels/handmade_model.h5"
qmodel = load_qmodel(model_path)

In [ ]:
qmodel.evaluate(X_train_val, y_train_val)
qmodel.evaluate(X_test, y_test)

# create hls model

In [ ]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(
    qmodel,
    granularity="name",
    default_precision="fixed<8,1>",
    default_reuse_factor=9000,
)

config["Model"]["Strategy"] = "Resource"

for layer in config["LayerName"].keys():
    config["LayerName"][layer]["Strategy"] = "Resource"

In [ ]:
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel,
    hls_config=config,
    output_dir="model_evaluation/hls4ml_prj",
    part="xc7z020-clg400-1",
    io_type="io_stream",
)
hls_model.compile()


In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
y_qkeras = qmodel.predict(np.ascontiguousarray(X_test))
y_hls = hls_model.predict(np.ascontiguousarray(X_test))
np.save("model_evaluation/y_qkeras.npy", y_qkeras)
np.save("model_evaluation/y_hls.npy", y_hls)

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

model_ref = keras.models.load_model(
    "/home/webphy/Desktop/dnn_processor/neural_network/handmade/saved_models/manual_test_4x/model.h5"
)
y_ref = model_ref.predict(X_test)

print(
    "Accuracy baseline:  {}".format(
        accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_ref, axis=1))
    )
)
print(
    "Accuracy quantized: {}".format(
        accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_qkeras, axis=1))
    )
)
print(
    "Accuracy hls4ml: {}".format(
        accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))
    )
)

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(y_test, y_ref, classes)
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(y_test, y_qkeras, classes, linestyle="--")
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(y_test, y_hls, classes, linestyle=":")

from matplotlib.lines import Line2D

lines = [Line2D([0], [0], ls="-"), Line2D([0], [0], ls="--"), Line2D([0], [0], ls=":")]
from matplotlib.legend import Legend

leg = Legend(
    ax,
    lines,
    labels=["baseline", "quantized", "hls4ml"],
    loc="lower right",
    frameon=False,
)
ax.add_artist(leg)

# profiling

In [ ]:
for layer in config["LayerName"].keys():
    config["LayerName"][layer]["Trace"] = True

hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel,
    hls_config=config,
    output_dir="model_evaluation/hls4ml_prj_2",
    part="xc7z020-clg400-1",
    io_type="io_stream",
)

hls4ml.model.profiling.numerical(model=qmodel, hls_model=hls_model, X=X_test[:])